# Gradient checks

It is best practice to do gradient checks before and after gradient-based optimization.

1. Find suitable tolerances to use during optimization. Importantly, test your gradients using the settings you will use later on.
2. At the optimum the values should be close to 0, except for parameters with active bounds. 
3. Gradient checks can help you identify inconsistencies and errors, especially when using custom gradient calculation or objectives.

Here we show, how to use the gradient check methods that are implemented in pyPESTO, using the finite differences (FD) method as a comparison. There is a trade-off between the quality of the approximation and numerical noise, so it is recommended to try different FD step sizes.


In [ ]:
import benchmark_models_petab as models
import numpy as np

import pypesto.optimize as optimize
import pypesto.petab

np.random.seed(2)

import pandas as pd
import seaborn as sns

from pypesto.examples import boehm

### Set up an example problem

Create the pypesto problem and a random vector of parameter values.  
Here, we use the startpoint sampling method to generate random parameter vectors.

In [ ]:
%%capture
pypesto_problem = boehm.problem
startpoints = pypesto_problem.get_startpoints(n_starts=4)

### Gradient check before optimization

Perform a gradient check at the location of one of the random parameter vectors. `check_grad` compares the gradients obtained by the finite differences (FD) method and the objective gradient. You can modify the finite differences step size via the argument `eps`.  

In [ ]:
pypesto_problem.objective.check_grad(
    x=startpoints[0],
    eps=1e-5,  # default
    verbosity=0,
)

Explanation of the gradient check result columns:

- `grad`: Objective gradient
- `fd_f`: FD forward difference
- `fd_b`: FD backward difference
- `fd_c`: Approximation of FD central difference (reusing the information from `fd_f` and `fd_b`)
- `fd_err`: Deviation between forward and backward differences `fd_f`, `fd_b`
- `abs_err`: Absolute error between `grad` and the central FD gradient `fd_c`
- `rel_err` Relative error between `grad` and the central FD gradient `fd_c`

If there are fixed parameters in your vector you might invoke an error due to the dimension mismatch. Use the helper method `Problem.get_reduced_vector` to get the reduced vector with only free (estimated) parameters.  
Here we set a smaller FD step size `eps = 1e-6` and observe that the errors change:

In [ ]:
parameter_vector = pypesto_problem.get_reduced_vector(startpoints[0])

pypesto_problem.objective.check_grad(
    x=parameter_vector,
    eps=1e-6,
    verbosity=0,
)

The method `check_grad_multi_eps` calls the `check_grad` method multiple times with different settings for the FD step size and reports the setting that results in the smallest error. 
You can supply a list of FD step sizes to be tested via the `multi_eps` argument (or use the default ones), and use the `label` argument to switch between the FD, or absolute or relative error.

In [ ]:
gc = pypesto_problem.objective.check_grad_multi_eps(
    x=parameter_vector,
    verbosity=0,
    label="rel_err",  # default
)

Use the pandas style methods to visualise the results of the gradient check, e.g.:

In [ ]:
def highlight_value_above_threshold(x, threshold=1):
    return ["color: darkorange" if xi > threshold else None for xi in x]


def highlight_gradient_check(gc: pd.DataFrame):
    return (
        gc.style.apply(
            highlight_value_above_threshold,
            subset=["fd_err"],
        )
        .background_gradient(
            cmap=sns.light_palette("purple", as_cmap=True),
            subset=["abs_err"],
        )
        .background_gradient(
            cmap=sns.light_palette("red", as_cmap=True),
            subset=["rel_err"],
        )
        .background_gradient(
            cmap=sns.color_palette("viridis", as_cmap=True),
            subset=["eps"],
        )
    )


highlight_gradient_check(gc)

There are consistently large discrepancies between forward and backward FD and a large relative error for the parameter `k_exp_hetero`.  

Ideally, all gradients would agree, but especially at not-so-smooth points of the objective, like (local) optima, large FD errors can occur.
It is recommended to check gradients over a lot of random points and check if there are consistently large errors for specific parameters.  

Below we perform a gradient check for another random point and observe small errors:

In [ ]:
parameter_vector = startpoints[1]

gc = pypesto_problem.objective.check_grad_multi_eps(
    x=parameter_vector,
    verbosity=0,
    label="rel_err",  # default
)
highlight_gradient_check(gc)

### Gradient check after optimization

Next, we do optimization and perform a gradient check at a local optimum.

In [ ]:
%%capture

result = optimize.minimize(
    problem=pypesto_problem,
    optimizer=optimize.ScipyOptimizer(),
    n_starts=4,
)

(Local) optima can be points with weird gradients. At a steep optimum, the `fd_err` is expected to be high.  

At the local optimum shown below, the `sd_pSTAT5B_rel` forward and backward FD have opposite signs and are quite large, resulting in a substantial `fd_err`. 

In [ ]:
# parameter vector at the local optimum, obtained from optimization
parameter_vector = pypesto_problem.get_reduced_vector(
    result.optimize_result[0].x
)

highlight_gradient_check(
    gc=pypesto_problem.objective.check_grad_multi_eps(
        x=parameter_vector,
        verbosity=0,
        label="rel_err",  # default
    )
)

### How to "fix" my gradients?

- Find suitable simulation tolerances.

Specific to the petab-amici-pipeline:

- Check the simulation logs for Warnings and Errors.
- Consider switching between forward and adjoint sensitivity algorithms.
